# Getting started

This tutorial introduces some basic concepts required to use this new version of PolSARpro in python.

It is important to note that the toolbox relies on Xarray. Here we only describe the basics required to run PolSARpro and refer the reader to https://docs.xarray.dev/en/stable/ for more details.

## Loading data

First we need to import some data. The demo dataset (TODO: host the data somewhere) has been prepared using SNAP and exported in the NetCDF-BEAM. NetCDF is the format recommended by Xarray as they the same data model.

Let's first open the data using the `open_netcdf_beam` function.

In [ ]:
from polsarpro.io import open_netcdf_beam
from pathlib import Path

# change to your data path
data_path = Path("/data/psp/test_files/SAN_FRANCISCO_ALOS1_slc.nc")
out_path =  Path("/data/psp/res")
S = open_netcdf_beam(data_path)

### Note

Currently, this function is able to recognize and open the following data types: Sinclair matrix `S`, covariance matrices `C3`, `C4` and `C2` as well as coherency matrices `T3` and `T4`. Preparation of polarimetric data using SNAP is out of the scope of this tutorial and may be found here: https://step.esa.int/docs/tutorials/S1TBX%20Polarimetry%20with%20Radarsat-2%20Tutorial_v2.pdf 

Once the data has been prepared in SNAP, it should be exported in NetCDF-BEAM format from the `File > Export` menu to be opened in PolSARpro.  

## Inspect data

The variable `S` has been recognized by the reader as a 2x2 Sinclair matrix. It has been imported in a data structure called `xarray.Dataset`. 
In a jupyter notebook it may be inspected as:

In [ ]:
print(S)

Each polarimetric channel is represented by a data variable and may be accessed using the following dot notation:

In [ ]:
# Access to the hh polarization channel
print(S.hh)

## Convert S to T3

Each variable is a dask array divided into smaller chunks, enabling parallel processing.
Please note that the data is not loaded into memory at this point. This is called lazy loading and allows to execute processing only when the result is required.

More information on lazy operations may be found here:https://docs.xarray.dev/en/latest/internals/internal-design.html#lazy-indexing-classes

To convert the S matrix to other types, we may use utility functions:

In [ ]:
from polsarpro.util import S_to_T3

T3 = S_to_T3(S)

Let's now look at this new variable:

In [ ]:
print(T3)

The output has been automatically converted to a new polarimetric type `T3` representing the coherency matrix. Now the elements of the matrix are accessed as:

In [ ]:
print(T3.m12)

For storage optimization, PolSARpro takes advantage of the Hermitian structure of the matrix and stores only diagonal and upper elements.

To ensure its algorithms are applied to valid polarimetric types, PolSARpro looks for a `poltype` "attribute" -- which is a kind of metadata attached to the dataset, and may be accessed as:

In [ ]:
T3.poltype

A human readable description may also be accessed as:

In [ ]:
T3.description

## Compute the results

To compute the actual result and load the data into memory, we may simply use:

In [ ]:
T3 = T3.compute()
print(T3)

## Crop, convert and save matrices

Now, each variable is a numpy array stored in memory.  

Lazy loading allows to define complex processing pipelines without storing intermediate variables in memory. It is even possible to write the result to disk without storing the whole data in memory. For instance one may define:

In [ ]:
from polsarpro.util import boxcar
from polsarpro.io import polmat_to_netcdf

# crop the data using the isel accessor, re-chunk to avoid errors when writing zarr
S_crop = S.isel(y=slice(5000, 12000)).chunk("auto")
T3 = S_to_T3(S_crop)

# save to a file with a custom function 
# that preserves chunks and handles complex valued matrices
polmat_to_netcdf(boxcar(T3, 5, 5), out_path / "T3_crop_box_5x5.nc")

## Apply the H/A/Alpha decomposition

Now let's apply the H/A/Alpha decomposition to our S matrix and
write the result to a NetCDF file:

In [ ]:
from polsarpro.decompositions import h_a_alpha
from dask.diagnostics import ProgressBar

# optional progress bar to monitor the process
with ProgressBar():
    h_a_alpha(S_crop, boxcar_size=[5, 5]).to_netcdf(out_path / "h_a_alpha.nc")

In [ ]:
import xarray as xr
# Let's open the file we have written
res = xr.open_dataset(out_path / "h_a_alpha.nc")
print(res)

As we can see, the new dataset has a specific `poltype` and stores the output parameters into labeled variables. As previously they can be accessed via the dot notation e.g. `res.alpha`.
We can now plot the outputs either using matplotlib or xarray plotting capabilities:

## Visualize the results

In [ ]:
res.alpha.plot.imshow()

In [ ]:
res.entropy.plot.imshow()

In [ ]:
res.anisotropy.plot.imshow()

Additionally, we can plot points in the H/Alpha plane and save the result as an image:

In [ ]:
from polsarpro.util import plot_h_alpha_plane
ax, fig = plot_h_alpha_plane(res, bins=400, min_pts=5)
fig.savefig(out_path / "h_a_alpha_plane.png")